In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/lib

__pycache__  utils.py


In [0]:
import os
import sys 
sys.path.append(os.getcwd() + "/drive/My Drive/Colab Notebooks/lib")
from utils import connectToDB, closeConnection
import pandas as pd

In [4]:
conn, curr = connectToDB()

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
curr.execute("SELECT subject_id from mimiciii.patients where expire_flag = 1;")

In [0]:
dead_patients = set()
for i in curr.fetchall():
  dead_patients.add(i[0])

In [7]:
len(dead_patients)

15759

In [0]:
curr.execute("SELECT subject_id, text, category from mimiciii.noteevents order by subject_id limit 200000;")

In [0]:
f = open('/content/drive/My Drive/Colab Notebooks/output3.csv', 'w')

In [0]:
while True:
    # Read the data
    df = pd.DataFrame(curr.fetchmany(10000))
    #print(df)
    # We are done if there are no data
    if len(df) == 0:
        break
    # Let's write to the file
    else:
        df.to_csv(f, header = None, index = False)

In [0]:
f.close()

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/outputs/output3.csv")

In [17]:
df.columns.values

array(['2',
       "Nursing Transfer note\n\n\nPt admitted to NICU for sepsis eval. Please see Attending\nnote for details regarding maternal history and delivery\ndetails.\n\nInfant stable in RA. RR 30-40's, sats 96-100%. LS clear/=.\nNo retractions noted. HR 140's. No murmur. Infant [** 5**], well\nperfused. BW 3865g. CBC and BC sent, pending at this time.\nInfant on 48 r/o sepsis with abx Amp and Gent. PIV placed in\nLeft hand, meds administered as ordered. D Stick 72. Infant\nstable for transfer to NBN. Continue to monitor for s/s of\nsepsis.\n\n\n",
       'Nursing/other'], dtype=object)

In [0]:
df.rename(columns = {"2" : "subject_id", "Nursing Transfer note\n\n\nPt admitted to NICU for sepsis eval. Please see Attending\nnote for details regarding maternal history and delivery\ndetails.\n\nInfant stable in RA. RR 30-40's, sats 96-100%. LS clear/=.\nNo retractions noted. HR 140's. No murmur. Infant [** 5**], well\nperfused. BW 3865g. CBC and BC sent, pending at this time.\nInfant on 48 r/o sepsis with abx Amp and Gent. PIV placed in\nLeft hand, meds administered as ordered. D Stick 72. Infant\nstable for transfer to NBN. Continue to monitor for s/s of\nsepsis.\n\n\n" : "text", "Nursing/other" : 'category'}, inplace = True)

In [19]:
df.head()

,subject_id,text,category
0,2,Neonatology Attending Triage Note\n\nBaby [**N...,Nursing/other
1,3,Admission Date: [**2101-10-20**] Discharg...,Discharge summary
2,3,PATIENT/TEST INFORMATION:\nIndication: S/P Car...,Echo
3,3,PATIENT/TEST INFORMATION:\nIndication: Pericar...,Echo
4,3,PATIENT/TEST INFORMATION:\nIndication: Left ve...,Echo


In [20]:
len(df)

199999

In [0]:
for i in range(len(df)):
  if df['category'][i] == 'Discharge summary' and df['subject_id'][i] in dead_patients:
    df.drop(i, inplace = True)

In [22]:
len(df)

197297

In [0]:
df['expire_flag'] = df['subject_id'].map(lambda x:1 if x in dead_patients else 0)

In [24]:
df.head()

,subject_id,text,category,expire_flag
0,2,Neonatology Attending Triage Note\n\nBaby [**N...,Nursing/other,0
2,3,PATIENT/TEST INFORMATION:\nIndication: S/P Car...,Echo,1
3,3,PATIENT/TEST INFORMATION:\nIndication: Pericar...,Echo,1
4,3,PATIENT/TEST INFORMATION:\nIndication: Left ve...,Echo,1
5,3,Sinus rhythm. A-V conduction delay. Slight Q-T...,ECG,1


In [0]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/outputs/notes,subjectid,expireflag.csv")